# Env

In [ ]:
%pip install -q langchain langchain_community langchain_chroma langchain_experimental langchain-text-splitters
%pip install -q langchain-groq langchain_openai
%pip install -q langchain-huggingface
%pip install -qU langchain-qdrant
%pip install -qU qdrant_client
%pip install -qU langchain-core
%pip install -qU langchain-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import getpass
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#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
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# RAG

In [ ]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
llm_drafter = ChatOpenAI(model="gpt-4o-mini", logprobs=True)
llm_verifier = ChatOpenAI(model="gpt-4o", logprobs=True)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
# Create a QdrantClient instance for Qdrant Cloud
qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

# Specify your collection name and query vector
collection_name = ""

In [ ]:
from sklearn.cluster import KMeans
from typing import Any
from collections import defaultdict
import numpy as np
def multi_perspective_sampling(
    k: int, retrieved_points: list[models.ScoredPoint], seed: int = 1399
) -> list[list[str]]:
    # Generate clusters
    print(f"Finding {k} clusters.")
    algo: Any = KMeans(n_clusters=k, random_state=seed)
    _vectors = [point.vector for point in retrieved_points]
    clusters: list[int] = algo.fit_predict(X=_vectors)

    # Unique clusters
    unique_clusters: set[int] = set(clusters)

    # Create a dictionary with the members of each cluster
    cluster_dict: defaultdict[int, list[int | None]] = defaultdict(list)
    for index, cluster in enumerate(clusters):
        cluster_dict[cluster].append(index)
    print(f"Clusters distribution: {dict(cluster_dict)}")

    # M subsets
    m: int = min(len(indices) for indices in cluster_dict.values())
    print(f"{m} document subsets will be created.")

    # Generate m unique subsets without replacement
    np.random.seed(seed=seed)
    subsets: list[list[str]] = []

    for _ in range(m):
        subset: list[int] = []
        for cluster in unique_clusters:
            chosen_element: int = np.random.choice(cluster_dict[cluster])
            subset.append(chosen_element)
            cluster_dict[cluster].remove(chosen_element)
        subset_documents = [
            retrieved_points[idx].payload.get("page_content") for idx in subset
        ]
        subsets.append(subset_documents)

    return subsets

In [ ]:
query = ""
query_vector = embeddings.embed_query(query)

In [ ]:
docs = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    with_vectors=True
)

In [ ]:
subsets = multi_perspective_sampling(k=2, retrieved_points=docs, seed = 123)

Finding 2 clusters.
Clusters distribution: {1: [0, 1, 2, 3, 4, 7], 0: [5, 6, 8, 9]}
4 document subsets will be created.


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate

prompt_draft = PromptTemplate.from_template("""Trả lời câu hỏi dựa trên các văn bản được cung cấp. Đồng thời cung cấp lý do đưa ra câu trả lời của bạn.
## Câu hỏi: {query}

## Văn bản cung cấp: {evidence}""")

class RagDraftingResponse(BaseModel):
    """Câu trả lời là lý do vì sao đưa ra câu trả lời như thế"""
    rationale: str = Field(description="Lý do đưa ra trả lời.")
    response: str = Field(description="Câu trả lời cho câu hỏi.")
llm_drafter_chain = (
    prompt_draft
    | llm_drafter.with_structured_output(RagDraftingResponse, include_raw=True)
)

In [ ]:
prompt_verify = PromptTemplate.from_template("""## Câu hỏi: {query}

## Câu trả lời: {response}

## Lý do: {rationale}

Lý do có đủ tốt để hỗ trợ cho câu trả lời không? (Có hoặc không)""")

llm_verifier_chain = (
    prompt_verify
    | llm_verifier
)

In [ ]:
res_draft = llm_drafter_chain.invoke({"query": query, "evidence": "\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(subsets[1], start=1)])})


In [ ]:
res_draft

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RVHGZjd2kWnyq56Q7bDSErdc', 'function': {'arguments': '{"rationale":"Cách mạng tư sản Pháp nổ ra trong bối cảnh kinh tế, xã hội khó khăn, với tình trạng áp bức nông dân và sự lạc hậu trong sản xuất nông nghiệp. Sự bất mãn của quần chúng nhân dân, đặc biệt là nông dân bị đối xử tàn nhẫn và sự phát triển của nền kinh tế hàng hóa đã tạo ra tiền đề cho cách mạng.","response":"Cách mạng tư sản Pháp nổ ra trong hoàn cảnh kinh tế khó khăn, xã hội bất công, nông dân bị áp bức và sự phát triển của nền kinh tế hàng hóa."}', 'name': 'RagDraftingResponse'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 385, 'total_tokens': 520, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e9627b5346', 'finish_reason': 'stop', 'logprobs': {'content': None, 'refusal': None}}, id='run-ff37655a-e93

In [ ]:
from tiktoken import Encoding, encoding_for_model, get_encoding
from statistics import mean

rag_verifications: list[tuple[str, float]] = []
for subset in subsets:
    res_draft = llm_drafter_chain.invoke({"query": query, "evidence": "\n".join([f"[{idx}] {doc}" for idx, doc in enumerate(subset, start=1)])})
    response = llm_verifier_chain.invoke({"query": query, "response": res_draft.response, "rationale": res_draft.rationale})
    encoder: Encoding = encoding_for_model(model_name=llm_verifier.model_name)
    cond: bool = encoder.encode(text=response.content.lower()) == encoder.encode(text="có.")
    p_yes: float = (
        np.exp(mean(token['logprob'] for token in response.response_metadata['logprobs']['content']))
        if cond
        else 0.0
    )  # Naive
    rag_verifications.append((res_draft.response, p_yes))


In [ ]:
best_answer: int = np.argmax(
    draft_score.p_yes for draft_score in rag_verifications
)

In [ ]:
rag_verifications[best_answer][0]

'Cách mạng tư sản Pháp nổ ra trong hoàn cảnh có sự phân chia giai cấp sâu sắc giữa giai cấp tư sản và giai cấp vô sản, với những điều kiện làm việc và sinh hoạt cực kỳ tồi tệ cho công nhân. Sự phát triển của chủ nghĩa tư bản và khủng hoảng kinh tế đã làm gia tăng mâu thuẫn xã hội, tạo ra bối cảnh thuận lợi cho sự nổ ra của cách mạng.'